In [2]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer, KafkaAdminClient
from kafka.admin import NewTopic
import redis
import time

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Kafka Admin setup for topic management
admin_client = KafkaAdminClient(bootstrap_servers='kafka:9092')

def delete_messages(topic):
    """Deletes all messages in a topic by recreating it."""
    try:
        admin_client.delete_topics([topic])  # Delete topic
        time.sleep(2)  # Wait for deletion
        admin_client.create_topics([NewTopic(name=topic, num_partitions=10, replication_factor=1)])  # Recreate topic
        print(f"🗑️ Deleted all messages in topic {topic} and recreated it.")
    except Exception as e:
        print(f"⚠️ Could not delete messages in topic {topic}: {e}")

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Consume messages from Kafka without using consumer groups."""
        topic = f'layer-{self.layer_id[-1]}'
        consumer = KafkaConsumer(
            topic,
            bootstrap_servers='kafka:9092',
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset='earliest',  # ✅ Always read messages from the beginning
            enable_auto_commit=True
        )

        print(f"🕒 Neuron {self.neuron_id} in {self.layer_id} waiting for activation on topic {topic}...")

        for message in consumer:
            print(f"✅ Neuron {self.neuron_id} received message: {message.value}")
            if 'layer' in message.value and message.value['layer'] == self.layer_id:
                print(f"🚀 Neuron {self.neuron_id} in {self.layer_id} activated!")
                break

        consumer.close()

        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        """Trigger neuron activations via Kafka, activate the next layer, then delete messages."""
        producer = KafkaProducer(
            bootstrap_servers='kafka:9092',
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )

        topic = f'layer-{self.layer_id[-1]}'
        activation_message = {'layer': self.layer_id}

        print(f"📤 Layer {self.layer_id} sending activation messages to topic {topic}...")

        try:
            producer.send(topic, value=activation_message)
            producer.flush()
            print(f"✅ Kafka message sent to {topic}")
        except Exception as e:
            print(f"❌ Kafka send failed: {e}")

        producer.close()

        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)

        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())

        # ✅ Activate the next layer via "activate-layer" topic before deleting messages
        producer = KafkaProducer(bootstrap_servers='kafka:9092',
                                 value_serializer=lambda v: json.dumps(v).encode('utf-8'))
        producer.send('activate-layer', {'layer': f'layer-{int(self.layer_id[-1]) + 1}'} if not self.is_final_layer else {'layer': 'final'})
        producer.flush()
        producer.close()

        # ✅ Delete all messages in the layer's topic AFTER activating the next layer
        delete_messages(topic)

        return outputs

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    prediction = forward_pass(network, image_np, i)
    print(f"Image {i} Prediction: {prediction}, Label: {label}")

NameError: name 'load_network' is not defined